In [1]:
import pandas as pd
import numpy as np
import os 
os.chdir("C:/Users/phu/OneDrive/Pictures")

# Calculate Customer Lifetime Value

In [2]:
df = pd.read_csv('Ecommerce_Data-1.csv')

In [3]:
df=df.drop(columns='Unnamed: 0')

In [4]:
df['Total_revenue'] = df['Quantity'] * df['UnitPrice']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81601 entries, 0 to 81600
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   InvoiceNo      81601 non-null  object 
 1   StockCode      81601 non-null  object 
 2   Description    81601 non-null  object 
 3   Quantity       81601 non-null  int64  
 4   UnitPrice      81601 non-null  float64
 5   CustomerID     81601 non-null  float64
 6   Country        81601 non-null  object 
 7   Date           81601 non-null  object 
 8   Hour           81601 non-null  int64  
 9   Total_revenue  81601 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 6.2+ MB


In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df = df[df['Quantity'] >0]

In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,Total_revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9,45.00
3,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13,16.50
4,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10,90.00


In [9]:
max_date = df['Date'].max()

In [10]:
df2 = df.groupby('CustomerID').agg({
    'Date':lambda x:(max_date-x.min()).days,
    'InvoiceNo':lambda x:len(x),
    'Quantity':lambda x:x.sum(),
    'Total_revenue':lambda x:x.sum()
})

In [11]:
def customer_moder(data):
    max_date =data['Date'].max()
    data = data.groupby('CustomerID').agg({
    'Date':lambda x:(max_date-x.min()).days,
    'InvoiceNo':lambda x:len(x),
    'Quantity':lambda x:x.sum(),
    'Total_revenue':lambda x:x.sum()
})
    return data
data = customer_moder(df)

In [12]:
data.columns = ['num_days','num_transaction','quanity','total_revenue']
data=data[data['quanity']>0]

In [13]:
data

,num_days,num_transaction,quanity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33
...,...,...,...,...
18280.0,277,3,9,52.75
18281.0,180,2,11,33.45
18282.0,126,4,11,67.85


# Getting the elements of CLTV Calculation

In [14]:
data['AOV'] = data['total_revenue']/data['num_transaction']

In [15]:
data

,num_days,num_transaction,quanity,total_revenue,AOV
CustomerID,,,,,
12347.0,367,48,623,1146.59,23.887292
12348.0,358,7,531,550.92,78.702857
12349.0,18,16,203,406.32,25.395000
12350.0,310,2,36,45.60,22.800000
12352.0,296,21,163,663.33,31.587143
...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333
18281.0,180,2,11,33.45,16.725000
18282.0,126,4,11,67.85,16.962500


## calculate the purchase frequency

In [16]:
purchase_freq = sum(data['num_transaction'])/len(data)
purchase_freq

19.496336101612115

## calculate the repeat_rate

In [17]:
repeat_rate = data[data['num_transaction'] >1].shape[0]/data.shape[0]

In [18]:
repeat_rate

0.9291646311675623

In [19]:
churn_rate = 1-repeat_rate

In [20]:
churn_rate

0.07083536883243768

In [21]:
data['profit_margin'] = data['total_revenue'] * .10

In [22]:
data

,num_days,num_transaction,quanity,total_revenue,AOV,profit_margin
CustomerID,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659
12348.0,358,7,531,550.92,78.702857,55.092
12349.0,18,16,203,406.32,25.395000,40.632
12350.0,310,2,36,45.60,22.800000,4.560
12352.0,296,21,163,663.33,31.587143,66.333
...,...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333,5.275
18281.0,180,2,11,33.45,16.725000,3.345
18282.0,126,4,11,67.85,16.962500,6.785


In [23]:
data['CLTV'] = ((data['AOV']* purchase_freq)/churn_rate)*.10

In [24]:
print("the median CLTV is",data['CLTV'].median())
print("the mean CLTV is",data['CLTV'].mean())

the median CLTV is 460.5410131773401
the mean CLTV is 747.7434697480141


In [25]:
data.sort_values(by='CLTV').reset_index()

,CustomerID,num_days,num_transaction,quanity,total_revenue,AOV,profit_margin,CLTV
0,12977.0,156,1,2,0.78,0.780000,0.078,21.468290
1,14661.0,26,1,1,0.85,0.850000,0.085,23.394931
2,16658.0,368,1,1,0.85,0.850000,0.085,23.394931
3,17962.0,142,3,3,2.99,0.996667,0.299,27.431703
4,17859.0,304,5,5,5.21,1.042000,0.521,28.679433
...,...,...,...,...,...,...,...,...
4089,12931.0,239,15,5590,9344.82,622.988000,934.482,17146.777994
4090,16333.0,298,11,7840,7944.40,722.218182,794.440,19877.934771
4091,17949.0,289,10,4565,7671.45,767.145000,767.145,21114.475728
4092,16000.0,2,5,2630,7948.30,1589.660000,794.830,43752.924786


# perdiction model for cltv 
# using multiple linear regression model

In [26]:
df['month-year'] = df['Date'].apply(lambda x:x.strftime("%b-%y"))

In [27]:
model = df[['CustomerID','InvoiceNo','Total_revenue','Quantity','UnitPrice','month-year']]

In [28]:
sales = model.pivot_table(index=['CustomerID'],columns=['month-year'],values='Total_revenue',aggfunc='sum',fill_value=0).reset_index()

In [29]:
sales['CLV'] = sales.iloc[:,2:].sum(axis =1)

In [30]:
sales

month-year,CustomerID,Apr-11,Aug-11,Dec-10,Dec-11,Feb-11,Jan-11,Jul-11,Jun-11,Mar-11,May-11,Nov-11,Oct-11,Sep-11,CLV
0,12347.0,113.60,205.56,140.00,57.00,0.0,155.95,0.00,114.00,0.00,0.00,0.00,360.48,0.00,1032.99
1,12348.0,137.00,0.00,332.16,0.00,0.0,41.76,0.00,0.00,0.00,0.00,0.00,0.00,40.00,413.92
2,12349.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,406.32,0.00,0.00,406.32
3,12350.0,0.00,0.00,0.00,0.00,45.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,45.60
4,12352.0,0.00,0.00,0.00,0.00,42.9,0.00,0.00,0.00,330.63,0.00,84.10,0.00,205.70,663.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4089,18280.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,52.75,0.00,0.00,0.00,0.00,52.75
4090,18281.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,33.45,0.00,0.00,0.00,0.00,0.00,33.45
4091,18282.0,0.00,12.75,0.00,55.10,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,67.85
4092,18283.0,33.86,0.00,0.00,28.62,42.5,36.15,10.96,40.31,0.00,22.20,85.48,9.61,15.85,291.68


In [31]:
x = sales[['Dec-11','Nov-11','Oct-11','Sep-11','Aug-11','Jul-11']]
y = sales[['CLV']]

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
x_train,x_test,y_train,y_test =train_test_split(x,y,random_state=0)
linear = LinearRegression()
linear.fit(x_train,y_train)
y_pred = linear.predict(x_test)

In [33]:
print(linear.intercept_)
print(linear.coef_)

[21.48367397]
[[1.12899377 1.17735188 1.77301549 1.34442525 2.2360908  2.19576538]]


In [34]:
coffiicients = linear.coef_.reshape(6,1)
a = pd.DataFrame(coffiicients,index=x_train.columns)

In [35]:
print("Intercept:", linear.intercept_)
print("Coefficients:", linear.coef_)

# Evaluate the model
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Intercept: [21.48367397]
Coefficients: [[1.12899377 1.17735188 1.77301549 1.34442525 2.2360908  2.19576538]]
Mean Absolute Error: 131.40959441617127
Mean Squared Error: 84139.94564737857
Root Mean Squared Error: 290.0688636296191
